### Imports

In [1]:
import os
import uuid
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, scoped_session

# Local
import models
import utils

Pandas utility functions

In [2]:
def get_relational_values(df: pd.DataFrame, column_name: str):
    """
    Get unique values from a specified column in a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame from which to extract values.
        column_name (str): The name of the column from which to extract unique values.

    Returns:
        list: A list containing the unique values from the specified column.
    """
    values = df[column_name].unique()
    return values.tolist()

In [3]:
def normalize_df(df: pd.DataFrame, normalized_data: list[dict]):
    """
    Normalize a DataFrame by replacing values according to the provided normalization rules.

    Args:
        df (pd.DataFrame): The DataFrame to be normalized.
        normalized_data (list[dict]): A list of dictionaries containing normalization rules.
            Each dictionary should have keys as old values to be replaced and corresponding values as new values.

    Returns:
        pd.DataFrame: The normalized DataFrame with values replaced according to the normalization rules.

    """
    for item in normalized_data:
        df = df.replace(item)
    return df

Database test connection

In [4]:
# Load enviroment variables
load_dotenv()

DATABASE_URI = os.getenv("DATABASE_URI")

# Connection
engine = create_engine(
    "postgresql://" + DATABASE_URI, connect_args={"sslmode": "allow"}
)

try:
    conn = engine.connect()
except Exception as e:
    print("Conection failed: ", e)
else:
    print("Connected! :D")

Connected! :D


Create database session

In [5]:
# Create models
models.Base.metadata.drop_all(engine)
models.Base.metadata.create_all(engine)

# Init Session
Session = scoped_session(sessionmaker(bind=engine))
session = Session()

Load dataset

In [6]:
df = pd.read_csv("clean_data.csv")

Define normalized rules

In [7]:
normalized_data = [
    {"Gender": {"Male": 0, "Female": 1}},
    {"Output": {"No": 0, "Yes": 1}},
    {"Feedback": {"Negative ": 0, "Positive": 1}},
]

relational_columns = [
    {"column": "Marital Status", "model": models.MatrialStatus},
    {"column": "Occupation", "model": models.OccupationStatus},
    {"column": "Monthly Income", "model": models.IncomeStatus},
    {"column": "Educational Qualifications", "model": models.EducationStatus},
]

Upload normalized values to db and get the id dict

In [8]:
for item in relational_columns:
    relational_values = get_relational_values(df, item["column"])
    nd_item = utils.upload_normalized_values(item["model"], relational_values, session)
    normalized_data.append({item["column"]: nd_item})

Normalice dataframe

In [9]:
df = normalize_df(df, normalized_data)
df.head()

/tmp/ipykernel_86576/1814428731.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(item)


,Age,Gender,Marital Status,Occupation,Monthly Income,Educational Qualifications,Family size,latitude,longitude,Pin code,Output,Feedback
0,20,1,1,1,1,1,4,12.9766,77.5993,560001,1,1
1,24,1,1,1,2,2,3,12.9770,77.5773,560009,1,1
2,22,0,1,1,2,1,3,12.9551,77.6593,560017,1,0
3,22,1,1,1,1,2,6,12.9473,77.5616,560019,1,1
4,22,0,1,1,2,1,4,12.9850,77.5533,560010,1,1


Upload dataframe to db

In [10]:
for index, row in df.iterrows():
    utils.create_register(
        db=session,
        model=models.User,
        id=uuid.uuid1(),
        id_matrial_status=row["Marital Status"],
        id_occupation_status=row["Occupation"],
        id_income_status=row["Monthly Income"],
        id_education_status=row["Educational Qualifications"],
        age=row["Age"],
        family_size=row["Family size"],
        latitude=row["latitude"],
        longitude=row["longitude"],
        gender=row["Gender"],
        output=row["Output"],
        feedback=row["Feedback"],
        zip_code=row["Pin code"],
    )

Extras

In [11]:
# Create order
utils.create_register(
    db=session,
    model=models.Orders,
    id=uuid.uuid1(),
    id_user="4017aea0-fcd5-11ee-8723-74563c469f4b",
    creation_date=datetime.now(),
    message = "Pedido urgente"
)

('ok', <models.Orders at 0x7f6a3407ede0>)

In [12]:
# Create products
utils.create_register(
    db=session,
    model=models.Products,
    id=uuid.uuid1(),
    name="Pizza Peperoni",
    description="Familiar"
)
utils.create_register(
    db=session,
    model=models.Products,
    id=uuid.uuid1(),
    name="Hamburguesa Hawaiana",
    description="Tocino extra"
)

('ok', <models.Products at 0x7f6a3407d520>)

In [14]:
utils.create_register(
    db=session,
    model=models.OrderProducts,
    id=uuid.uuid1(),
    id_order="8f7fa11e-fcd5-11ee-8723-74563c469f4b",
    id_product="91fdea36-fcd5-11ee-8723-74563c469f4b",
    amount="1"
)
utils.create_register(
    db=session,
    model=models.OrderProducts,
    id=uuid.uuid1(),
    id_order="8f7fa11e-fcd5-11ee-8723-74563c469f4b",
    id_product="92310ad8-fcd5-11ee-8723-74563c469f4b",
    amount="2"
)

('ok', <models.OrderProducts at 0x7f6a341f6570>)